there's a lot of non-bio relevant hits, so let's filter them out

In [3]:
import os
from pathlib import Path
root = Path('/Users/user/Coding/EV-D68-3C-protease/')
os.chdir(root)

In [14]:
from rdkit import Chem
import pymol2

neighbors = []
hits = []
with Chem.SDMolSupplier('hits.sdf') as suppl:
    with pymol2.PyMOL() as pymol:
        pymol.cmd.load('reference.pdb', 'ref')
        pymol.cmd.fetch('2B0F')
        pymol.cmd.align('2B0F', 'ref')
        pymol.cmd.delete('ref')
        pymol.cmd.remove('%2B0F and not chain B')
        pymol.cmd.remove('solvent')
        for mol in suppl:
            hits.append(mol)
            pymol.cmd.read_molstr(Chem.MolToMolBlock(mol), 'mol')
            n_neighbors = pymol.cmd.count_atoms('%2B0F within 4 of mol')
            pymol.cmd.delete('mol')
            if n_neighbors > 0:
                neighbors.append(mol)

 ExecutiveLoad-Detail: Detected mmCIF


In [15]:
len(neighbors), len(hits)

(19, 104)

In [16]:
# Save

with Chem.SDWriter('filtered_hits.sdf') as sdf:
    for mol in neighbors:
        if mol is None:
            continue
        sdf.write(mol)

In [23]:
from rdkit.Chem import Draw

with open('filtered_hits.png', 'wb') as fh:
    fh.write(Draw.MolsToGridImage(neighbors).data)